# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':')
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
document = ET.parse( './data/mondial_database.xml' )

In [6]:
for country in document.getroot().iterfind('country'):
    if country.find('infant_mortality') == None:
        continue
    #print(country.find('name').text + " " + country.find('infant_mortality').text)

In [7]:
import pandas as pd

In [8]:
infant_mortality = pd.DataFrame(columns=['Country', 'Infant Mortality'])
infant_mortality

,Country,Infant Mortality


In [9]:
#extract and append the rows of interest to the infant_mortality dataframe 
#we can use a list comprehension together with iterfind to iterate through desired entries
#some countries do not have an infant mortality entry. skip those 
for country in document.getroot().iterfind('country'):
    if country.find('infant_mortality') == None:
        continue
    temp = pd.DataFrame([[country.find('name').text,country.find('infant_mortality').text]],columns=['Country','Infant Mortality'])
    infant_mortality = infant_mortality.append(temp)

In [10]:
infant_mortality['Infant Mortality'] = infant_mortality['Infant Mortality'].astype(float)

In [36]:
infant_mortality.sort_values('Infant Mortality').head(10)

,Country,Infant Mortality
0,Monaco,1.81
0,Japan,2.13
0,Bermuda,2.48
0,Norway,2.48
0,Singapore,2.53
0,Sweden,2.60
0,Czech Republic,2.63
0,Hong Kong,2.73
0,Macao,3.13
0,Iceland,3.15


In [12]:
city_population = pd.DataFrame(columns=['City', 'Population'])
city_population

,City,Population


In [13]:
for country in document.getroot().iterfind('country'):
    for city in country.iterfind('city'):
        if city.find('population') == None:
            continue
        temp = pd.DataFrame([[city.find('name').text,city.findall('population')[len(city.findall('population'))-1].text]],columns=['City','Population'])
        city_population = city_population.append(temp)
        

In [38]:
city_population['Population'] = city_population['Population'].astype(int)
city_population.sort_values('Population', ascending=False).head(10)

,City,Population
0,Seoul,9708483
0,Al Qahirah,8471859
0,Bangkok,7506700
0,Hong Kong,7055071
0,Ho Chi Minh,5968384
0,Singapore,5076700
0,Al Iskandariyah,4123869
0,New Taipei,3939305
0,Busan,3403135
0,Pyongyang,3255288


In [15]:
#testing expression to extract percentage for an ethnic group
(int(document.getroot().find('country').find('ethnicgroup').attrib['percentage'])/100)

0.95

In [16]:
#population is listed for several years for each country. this expression extracts the last listed (most recent) population estimate
int(document.getroot().find('country').findall('population')[len(document.getroot().find('country').findall('population'))-1].text)

2800138

In [17]:
ethnic_groups = pd.DataFrame(columns=['Ethnicity','Percentage','Country Population'])
ethnic_groups

,Ethnicity,Percentage,Country Population


In [18]:
for country in document.getroot().iterfind('country'):
    for ethnicity in country.iterfind('ethnicgroup'):
        if country.find('population') == None:
            continue
        temp = pd.DataFrame([[ethnicity.text,float(ethnicity.attrib['percentage'])/100,country.findall('population')[len(country.findall('population'))-1].text]],
                           columns=['Ethnicity','Percentage','Country Population'])
        ethnic_groups = ethnic_groups.append(temp)


In [19]:
ethnic_groups['Percentage'] = ethnic_groups['Percentage'].astype(float)
ethnic_groups['Country Population'] = ethnic_groups['Country Population'].astype(int)
ethnic_groups['Ethnic Population'] = ethnic_groups['Percentage']*ethnic_groups['Country Population']

In [20]:
#lists ethnic populations in each country. (countries not listed--but this is why there are duplicate entries)
ethnic_groups['Ethnic Population'] = ethnic_groups['Ethnic Population'].astype(int)
ethnic_groups.head(10)

,Ethnicity,Percentage,Country Population,Ethnic Population
0,Albanian,0.950,2800138,2660131
0,Greek,0.030,2800138,84004
0,Greek,0.930,10816286,10059145
0,Macedonian,0.642,2059794,1322387
0,Albanian,0.252,2059794,519068
0,Turkish,0.039,2059794,80331
0,Gypsy,0.027,2059794,55614
0,Serb,0.018,2059794,37076
0,Serb,0.829,7120666,5903032
0,Montenegrin,0.009,7120666,64085


In [21]:
#groupby and sum ethnic populations to return greatest ethnic populations over all countries
ethnic_groups[['Ethnicity','Ethnic Population']].groupby(['Ethnicity',]).sum().sort_values('Ethnic Population',ascending=False).head(10)

,Ethnic Population
Ethnicity,
Han Chinese,1245058800
Indo-Aryan,871815583
European,494872201
African,318325104
Dravidian,302713744
Mestizo,157734349
Bengali,146776916
Russian,131856989
Japanese,126534212


In [22]:
ethnic_groups['Ethnic Population'].sum()

5960384709

In [23]:
#testing expressions to extract country, river length, country
print(document.getroot().find('river').find('name').text)
country_abbrev = document.getroot().find('river').find('source').attrib['country']
print(int(document.getroot().find('river').find('length').text))

for country in document.getroot().iterfind('country'):
    if country.attrib['car_code'] == country_abbrev:
        print(country.find('name').text)

Thjorsa
230
Iceland


In [24]:
rivers = pd.DataFrame(columns=['Country','River Name','Length'])
rivers

,Country,River Name,Length


In [25]:
#append all rivers (and additional desired information) into a dataframe
for river in document.getroot().iterfind('river'):
    if river.find('length')==None:
        continue
    country_abbrev = river.find('source').attrib['country']
    river_name = river.find('name').text
    length = float(river.find('length').text)
    for country in document.getroot().iterfind('country'):
        if country.attrib['car_code'] == country_abbrev:
            country_name = country.find('name').text
    temp = pd.DataFrame([[country_name,river_name,length]],columns=['Country','River Name','Length'])
    rivers = rivers.append(temp)

In [26]:
rivers.sort_values('Length',ascending=False).head(1)

,Country,River Name,Length
0,Peru,Amazonas,6448.0


In [27]:
print(document.getroot().find('lake').find('name').text)
print(document.getroot().find('lake').find('area').text)
country_abbrev = document.getroot().find('lake').attrib['country']
for country in document.getroot().iterfind('country'):
    if country.attrib['car_code'] == country_abbrev:
        print(country.find('name').text)

Inari
1040
Finland


In [28]:
lakes_df1 = pd.DataFrame(columns=['Country','Lake','Area'])
lakes_df1

,Country,Lake,Area


In [29]:
for lake in document.getroot().iterfind('lake'):
    if lake.find('area') == None:
        continue
    lake_name = lake.find('name').text
    lake_area = float(lake.find('area').text)
    country_abbrev = lake.attrib['country']
    for country in document.getroot().iterfind('country'):
        if country.attrib['car_code'] == country_abbrev:
            country_name = country.find('name').text
    temp = pd.DataFrame([[country_name,lake_name,lake_area]],columns=['Country','Lake','Area'])
    lakes_df1 = lakes_df1.append(temp)

In [30]:
lakes_df1.sort_values('Area',ascending=False).head(1)

,Country,Lake,Area
0,Iran,Caspian Sea,386400.0


In [31]:
print(document.getroot().find('airport').find('name').text)
print(document.getroot().find('airport').find('elevation').text)
print(document.getroot().find('airport').attrib['country'])

Herat
977
AFG


In [32]:
airports = pd.DataFrame(columns=['Country','Airport','Elevation'])
airports

,Country,Airport,Elevation


In [33]:
for airport in document.getroot().iterfind('airport'):
    if airport.find('elevation')==None:
        continue
    airport_name = airport.find('name').text
    elevation = airport.find('elevation').text
    country_abbrev = airport.attrib['country']
    for country in document.getroot().iterfind('country'):
        if country.attrib['car_code'] == country_abbrev:
            country_name = country.find('name').text
    temp = pd.DataFrame([[country_name,airport_name,elevation]],columns=['Country','Airport','Elevation'])
    airports = airports.append(temp)
            

In [34]:
airports['Elevation'] = airports['Elevation'].astype(float)

In [35]:
airports.sort_values('Elevation',ascending=False).head(1)

,Country,Airport,Elevation
0,Bolivia,El Alto Intl,4063.0
